### PgLearn Data

In [1]:
import h5py
import pandas as pd
import numpy as np
import json

In [24]:
data_dir = "/Users/albanpuech/benchmark_data/PGLearn-Small-118_ieee-nminus1"

In [28]:
# Input dataset for Pd, Qd: (N, L)
input_data = data_dir + "/input.h5"
input_dataset = h5py.File(input_data, "r")

# Primal dataset for Pg, Qg: (N, G), Va, Vm: (N, N)
primal_data = data_dir + "/primal.h5"
primal_dataset = h5py.File(primal_data, "r")

# Case file
with open(
    data_dir + "/case.json", "r"
) as f:
    case_data = json.load(f)

pg = np.array(primal_dataset["pg"])
qg = np.array(primal_dataset["qg"])
vm = np.array(primal_dataset["vm"])
va = np.array(primal_dataset["va"])
Pd = np.array(input_dataset["pd"])
qd = np.array(input_dataset["qd"])

n_scenarios = va.shape[0]
n_buses = va.shape[1]

n_gens = pg.shape[1]
n_loads = qd.shape[1]


buses_with_gens = [i for i in range(n_buses) if case_data['data']['bus_gens'][i]]
buses_with_loads = [i for i in range(n_buses) if case_data['data']['bus_loads'][i]]

assert len(buses_with_gens) == n_gens, "Number of generators is not correct"
assert len(buses_with_loads) == n_loads, "Number of loads is not correct"

assert max([len(i) for i in case_data['data']['bus_gens']]) == 1, "There should be only one generator per bus"
assert max([len(i) for i in case_data['data']['bus_loads']]) == 1, "There should be only one load per bus"

# assert gen_idx are consecutive
assert (np.concatenate(case_data['data']['bus_gens']).astype(int)-1 == list(range(n_gens))).all(), "gen_idx are not consecutive"
assert (np.concatenate(case_data['data']['bus_loads']).astype(int)-1 == list(range(n_loads))).all(), "load_idx are not consecutive"

pg_all = np.zeros((n_scenarios, n_buses))
qg_all = np.zeros((n_scenarios, n_buses))
pd_all = np.zeros((n_scenarios, n_buses))
qd_all = np.zeros((n_scenarios, n_buses))

pd_all[:, buses_with_loads] = Pd
qd_all[:, buses_with_loads] = qd

pg_all[:, buses_with_gens] = pg
qg_all[:, buses_with_gens] = qg


df_pglearn = pd.DataFrame(
    {
                "scenario": np.repeat(np.arange(n_scenarios), n_buses),

        "bus": np.tile(np.arange(n_buses), n_scenarios),
        "Pd": pd_all.flatten()*100.0,
        "Qd": qd_all.flatten()*100.0,
        "Pg": pg_all.flatten()*100.0,
        "Qg": qg_all.flatten()*100.0,
        "Vm": vm.flatten(),
        "Va": np.rad2deg(va.flatten()),
    })





In [32]:
# write df to csv
df_pglearn.to_csv(
    data_dir + "/pglearn.csv",
    index=False,
)



In [5]:
df = pd.read_csv("/Users/albanpuech/benchmark_data/pglearn/pglearn.csv")

In [22]:
# compare df and df_pglearn
print(df.mean())
print(df_pglearn.mean())

assert np.allclose(df.mean(), df_pglearn.mean()), "df and df_pglearn are not the same"

scenario    399993.500000
bus             58.500000
Pd              35.951141
Qd              12.186679
Pg              37.134535
Qg               7.015029
Vm               1.033266
Va             -10.508916
dtype: float64
scenario    399993.500000
bus             58.500000
Pd              35.951141
Qd              12.186679
Pg              37.134535
Qg               7.015029
Vm               1.033267
Va             -10.508917
dtype: float64
